# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0435, -0.1743, -0.0328,  0.0465,  0.0752,  0.0694,  0.1195,  0.0587,
          0.0381,  0.0319],
        [ 0.0119, -0.2532,  0.0167, -0.0171,  0.1693,  0.0286,  0.0719,  0.0257,
          0.0438,  0.1290]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-2.9990e-02, -4.6236e-02, -4.4129e-02, -3.0950e-01,  1.1084e-01,
         -1.6770e-01, -1.4034e-01,  3.1303e-02,  7.8324e-02, -9.9552e-02],
        [-7.5197e-02, -9.8461e-02, -1.6198e-01, -3.7348e-01,  3.0911e-02,
         -1.7290e-01, -4.8249e-02,  6.1693e-02, -1.7278e-04, -1.2514e-01]],
       grad_fn=<AddmmBackward0>)

In [4]:
net._modules

OrderedDict([('hidden', Linear(in_features=20, out_features=256, bias=True)),
             ('out', Linear(in_features=256, out_features=10, bias=True))])

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1296, -0.0557,  0.1170, -0.0532,  0.0182,  0.0192, -0.1107, -0.1775,
         -0.0353, -0.1188],
        [ 0.2459,  0.0037,  0.1989, -0.0154,  0.0638, -0.0092, -0.1410, -0.1916,
          0.0248, -0.0236]], grad_fn=<AddmmBackward0>)

In [6]:
net._modules

OrderedDict([('0', Linear(in_features=20, out_features=256, bias=True)),
             ('1', ReLU()),
             ('2', Linear(in_features=256, out_features=10, bias=True))])

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0312, grad_fn=<SumBackward0>)

In [8]:
net._modules

OrderedDict([('linear', Linear(in_features=20, out_features=20, bias=True))])

混合搭配各种组合块的方法

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1082, grad_fn=<SumBackward0>)

In [11]:
chimera._modules

OrderedDict([('0',
              NestMLP(
                (net): Sequential(
                  (0): Linear(in_features=20, out_features=64, bias=True)
                  (1): ReLU()
                  (2): Linear(in_features=64, out_features=32, bias=True)
                  (3): ReLU()
                )
                (linear): Linear(in_features=32, out_features=16, bias=True)
              )),
             ('1', Linear(in_features=16, out_features=20, bias=True)),
             ('2',
              FixedHiddenMLP(
                (linear): Linear(in_features=20, out_features=20, bias=True)
              ))])

## 练习

In [13]:
class MySeries(nn.Module):
  def __init__(self, net1, net2):
    super().__init__()
    self.net1 = net1
    self.net2 = net2
    
  def forward(self, X):
    return torch.cat((self.net1(X), self.net2(X)), dim=1)

In [15]:
series = MySeries(nn.Linear(20, 5), nn.Linear(20,6))

series(X)

tensor([[ 0.1003, -0.4914,  0.1773, -0.0271,  0.1074,  0.3826, -0.3979,  0.1856,
          0.0379, -0.1527, -0.0068],
        [ 0.0293, -0.2778,  0.3792, -0.1102,  0.2133,  0.3731, -0.4541,  0.3632,
          0.2616, -0.3192, -0.0031]], grad_fn=<CatBackward0>)

In [16]:
series._modules

OrderedDict([('net1', Linear(in_features=20, out_features=5, bias=True)),
             ('net2', Linear(in_features=20, out_features=6, bias=True))])